In [15]:
import csv
from pathlib import Path
from fast_forward.ranking import Ranking
from fast_forward.index import InMemoryIndex
from fast_forward.encoder import TCTColBERTQueryEncoder as TCTColBERTQueryEncoderFF
from ir_measures import read_trec_qrels, calc_aggregate, nDCG, RR, R, AP, P
from numpy import arange
import csv

In [16]:
sparse_ranking_2019 = Ranking.from_file(Path("../index/dev/testing/msmarco-passage-test2019-sparse10000.txt"))
sparse_ranking_2019.cut(5000)
sparse_ranking_2020 = Ranking.from_file(Path("../index/dev/testing/msmarco-passage-test2020-sparse10000.txt"))
sparse_ranking_2020.cut(5000)

index = InMemoryIndex.from_disk("../index/dev/testing/ffindex_passage_2019_2020.pkl",encoder=TCTColBERTQueryEncoderFF("castorini/tct_colbert-msmarco"))

Some weights of the model checkpoint at castorini/tct_colbert-msmarco were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
with open(
    "../index/dev/testing/msmarco-test2020-queries.tsv",
    encoding="utf-8",
    newline=""
) as fp:
    queries = {q_id: q for q_id, q in csv.reader(fp, delimiter="\t")}
print(f"loaded {len(queries)} queries")

alpha = arange(0,1.01,0.05)
eta = range(5,106,5)
alpha = [0,0.85,1]
eta = [60]
result_cc = index.get_scores(
    sparse_ranking_2020,
    queries,
    method = "CC",
    alpha = alpha,
    cutoff=10,
    early_stopping=False,
    normalize = "Z" # None, "MM", "TMM", "Z"
)
#result_rrf = index.get_scores(
#    sparse_ranking_2020,
#    queries,
#    method = "RRF",
#    eta = eta,
#    cutoff=10,
#    early_stopping=False
#)

loaded 200 queries
custom FFI version


100%|██████████| 54/54 [00:00<00:00, 55.57it/s]


w:  0
w:  0.85
w:  1


### CC - 2020 (test set)

In [18]:
qrels = list(read_trec_qrels("../index/dev/testing/2020qrels-pass.txt"))
with open(r'2020_test_results_Z.csv','a') as f1: # need "a" and not w to append to a file, if not will overwrite
    writer=csv.writer(f1, delimiter=',',lineterminator='\n',)
    row = ["fusion_function","dataset","alpha","AP@10","AP@100","nDCG@10","nDCG@100"]
    writer.writerow(row)
    for a in alpha:
        f = calc_aggregate([AP(rel=3)@10,AP@100,nDCG@10,nDCG@100], qrels, result_cc[a].run)
        row = ["CC","test2020",a,f[AP(rel=3)@10],f[AP@100],f[nDCG@10],f[nDCG@100]]
        writer.writerow(row)


### RRF - 2020 (test set)

In [ ]:

with open(r'2020_test_results.csv','a') as f1: # need "a" and not w to append to a file, if not will overwrite
    writer=csv.writer(f1, delimiter=',',lineterminator='\n',)
    for n in eta:
        f = calc_aggregate([AP(rel=3)@10,AP@100,nDCG@10,nDCG@100], qrels, result_rrf[n].run)
        row = ["RRF","test2020",n,f[AP(rel=3)@10],f[AP@100],f[nDCG@10],f[nDCG@100]]
        writer.writerow(row)